In [2]:

import pandas as pd

ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'])


print(ratings.head())

   user_id  movie_id  rating  timestamp
0      196       242       3  881250949
1      186       302       3  891717742
2       22       377       1  878887116
3      244        51       2  880606923
4      166       346       1  886397596


In [6]:
user_movie_matrix = ratings.pivot_table(index='user_id', columns='movie_id', values='rating')


user_movie_matrix.fillna(0, inplace=True)


user_similarity = cosine_similarity(user_movie_matrix)


user_similarity_df = pd.DataFrame(user_similarity, index=user_movie_matrix.index, columns=user_movie_matrix.index)


print(user_similarity_df.head())

user_id       1         2         3         4         5         6         7    \
user_id                                                                         
1        1.000000  0.166931  0.047460  0.064358  0.378475  0.430239  0.440367   
2        0.166931  1.000000  0.110591  0.178121  0.072979  0.245843  0.107328   
3        0.047460  0.110591  1.000000  0.344151  0.021245  0.072415  0.066137   
4        0.064358  0.178121  0.344151  1.000000  0.031804  0.068044  0.091230   
5        0.378475  0.072979  0.021245  0.031804  1.000000  0.237286  0.373600   

user_id       8         9         10   ...       934       935       936  \
user_id                                ...                                 
1        0.319072  0.078138  0.376544  ...  0.369527  0.119482  0.274876   
2        0.103344  0.161048  0.159862  ...  0.156986  0.307942  0.358789   
3        0.083060  0.061040  0.065151  ...  0.031875  0.042753  0.163829   
4        0.188060  0.101284  0.060859  ...  0.052107

In [19]:
movie_user_matrix = user_movie_matrix.T

movie_similarity = cosine_similarity(movie_user_matrix)

movie_similarity_df = pd.DataFrame(movie_similarity, index=movie_user_matrix.index, columns=movie_user_matrix.index)

print(movie_similarity_df.head())

movie_id      1         2         3         4         5         6     \
movie_id                                                               
1         1.000000  0.402382  0.330245  0.454938  0.286714  0.116344   
2         0.402382  1.000000  0.273069  0.502571  0.318836  0.083563   
3         0.330245  0.273069  1.000000  0.324866  0.212957  0.106722   
4         0.454938  0.502571  0.324866  1.000000  0.334239  0.090308   
5         0.286714  0.318836  0.212957  0.334239  1.000000  0.037299   

movie_id      7         8         9         10    ...      1673  1674  \
movie_id                                          ...                   
1         0.620979  0.481114  0.496288  0.273935  ...  0.035387   0.0   
2         0.383403  0.337002  0.255252  0.171082  ...  0.000000   0.0   
3         0.372921  0.200794  0.273669  0.158104  ...  0.000000   0.0   
4         0.489283  0.490236  0.419044  0.252561  ...  0.000000   0.0   
5         0.334769  0.259161  0.272448  0.055453  ...  0.

In [20]:
def recommend_movies_user_based(user_id, user_movie_matrix, user_similarity_df, n_recommendations=5):
    similar_users = user_similarity_df[user_id].sort_values(ascending=False)

    similar_users_ratings = user_movie_matrix.loc[similar_users.index]

    weighted_ratings = similar_users_ratings.T.dot(similar_users) / similar_users.sum()

    user_ratings = user_movie_matrix.loc[user_id]
    recommendations = weighted_ratings[user_ratings.isna()].sort_values(ascending=False)
   
    return recommendations.head(n_recommendations)




In [ ]:
def recommend_movies_item_based(user_id, user_movie_matrix, movie_similarity_df, n_recommendations=5):
    user_ratings = user_movie_matrix.loc[user_id]

    rated_movies = user_ratings[user_ratings.notna()]

    weighted_ratings = movie_similarity_df[rated_movies.index].dot(rated_movies) / movie_similarity_df[rated_movies.index].sum(axis=1)

    recommendations = weighted_ratings[user_ratings.isna()].sort_values(ascending=False)

    return recommendations.head(n_recommendations)


In [21]:
user_id = 1
user_based_recommendations = recommend_movies_user_based(user_id, user_movie_matrix, user_similarity_df)

print(f"Recomendações baseadas em usuários para o usuário {user_id}:")
print(user_based_recommendations)

item_based_recommendations = recommend_movies_item_based(user_id, user_movie_matrix, movie_similarity_df)

print(f"\nRecomendações baseadas em itens para o usuário {user_id}:")
print(item_based_recommendations)


Recomendações baseadas em usuários para o usuário 1:
Series([], dtype: float64)

Recomendações baseadas em itens para o usuário 1:
Series([], dtype: float64)
